# SQL EMBEDDING SELECTION EXPERIMENT

In [ ]:
import os
import json
import nest_asyncio
from tqdm import tqdm
from utils.prompts.prompt_builder import prompt_factory
from utils.data.data_builder import load_data
from utils.llm.ask_llm import run_llm
from third_party.spider_eval.evaluation import evaluate_spider
from utils.data.post_process import save_results
from utils.prompts.prompt_builder import get_openai_key

EMBEDDING = 's2593817/sft-sql-embedding'

In [1]:
from openai import OpenAI

# Set the environment variable to provide access to the OpenAI API (set in utils/parameters.py)
os.environ['OPENAI_API_KEY'] = get_openai_key()

# Create the OpenAI client
client = OpenAI()

In [3]:
path_data = "benchmarks"

data = load_data("spider", path_data)

## SQL Fine-Tuned Embedding Selector Spider 1-Shot Experiment

In [4]:
prompt = prompt_factory(k_shot = 1, repr_type= "SQL", example_format= "QA", selector_type= "EMBEDSQL", embedding_model= EMBEDDING)(data=data, tokenizer="gpt-3.5-turbo")

In [5]:
# Format all Spider test questions for 1-Shot GPT-3.5-Turbo Text-to-SQL conversion
questions = list()

for i, question_json in enumerate(tqdm(getattr(data, "get_test_json")()), start=1):
    question_format = prompt.format(index=i,
                                    target=question_json,
                                    max_seq_len=2048,
                                    max_ans_len=200,
                                    scope_factor=1,
                                    cross_domain=False)
    questions.append(question_format)

100%|██████████| 1034/1034 [04:19<00:00,  3.98it/s]


In [6]:
# Define the task parameters for saving the questions for the Spider baseline experiment

args = {
    "data_type": "spider",
    "split": "test",
    "tokenizer": "gpt-3.5-turbo",
    "max_seq_length": 2048,
    "prompt_repr": "SQL",
    "k-shot": 1,
    "example_type": "QA",
    "selector_type": "EMBEDPRED",
    "embedding_model": EMBEDDING
}

# Define the task object for saving the questions for the Spider baseline experiment

task = {
        "args": args,
        "questions": questions
    }

OUT_DIR = os.path.join("chapter-4","results", "sql-embedding-sft-experiments", "1-shot")
os.makedirs(OUT_DIR, exist_ok =True)

PROMPTS_FILE = os.path.join(OUT_DIR, "sql-embedding-sft-prompts-1.json")
json.dump(task, open(PROMPTS_FILE, "w"), indent=4)

In [7]:
OUT_DIR = os.path.join("chapter-4","results", "sql-embedding-sft-experiments", "1-shot")

PROMPTS_FILE = os.path.join(OUT_DIR, "sql-embedding-sft-prompts-1.json")

RESPONSES_FILE = os.path.join(OUT_DIR, "sql-embedding-sft-responses-1.txt")

run_llm(PROMPTS_FILE, RESPONSES_FILE, model="gpt-3.5-turbo")

100%|██████████| 1034/1034 [15:50<00:00,  1.09it/s] 


In [8]:
nest_asyncio.apply()

OUT_DIR = os.path.join("chapter-4","results", "sql-embedding-sft-experiments", "1-shot")

PROMPTS_FILE = os.path.join(OUT_DIR, "sql-embedding-sft-prompts-1.json")
RESPONSES_FILE = os.path.join(OUT_DIR, "sql-embedding-sft-responses-1.txt")
RESULTS_FILE = os.path.join(OUT_DIR,"sql-embedding-sft-results-1.json")

results = evaluate_spider(gold="benchmarks/spider/dev_gold.sql", pred=RESPONSES_FILE, db="benchmarks/spider/databases", table="benchmarks/spider/tables.json")

save_results(PROMPTS_FILE, RESPONSES_FILE, RESULTS_FILE, results)

  0%|          | 0/1 [00:00<?, ?it/s]


                     easy                 medium               hard                 extra                all                 
count                248                  446                  174                  166                  1034                
=====================   EXECUTION ACCURACY     =====================
execution            0.915                0.843                0.678                0.536                0.783               


## SQL Fine-Tuned Embedding Selector Spider 3-shot Experiment

In [9]:
prompt = prompt_factory(k_shot = 3, repr_type= "SQL", example_format= "QA", selector_type= "EMBEDSQL", embedding_model= EMBEDDING)(data=data, tokenizer="gpt-3.5-turbo")

In [10]:
# Format all Spider test questions for 3-Shot GPT-3.5-Turbo Text-to-SQL conversion
questions = list()

for i, question_json in enumerate(tqdm(getattr(data, "get_test_json")()), start=1):
    question_format = prompt.format(index=i,
                                    target=question_json,
                                    max_seq_len=2048,
                                    max_ans_len=200,
                                    scope_factor=1,
                                    cross_domain=False)
    questions.append(question_format)

100%|██████████| 1034/1034 [05:45<00:00,  2.99it/s]


In [11]:
# Define the task parameters for saving the questions for the Spider baseline experiment

args = {
    "data_type": "spider",
    "split": "test",
    "tokenizer": "gpt-3.5-turbo",
    "max_seq_length": 2048,
    "prompt_repr": "SQL",
    "k-shot": 3,
    "example_type": "QA",
    "selector_type": "EMBEDPRED",
    "embedding_model": EMBEDDING
}

# Define the task object for saving the questions for the Spider baseline experiment

task = {
        "args": args,
        "questions": questions
    }

OUT_DIR = os.path.join("chapter-4","results", "sql-embedding-sft-experiments", "3-shot")
os.makedirs(OUT_DIR, exist_ok =True)

PROMPTS_FILE = os.path.join(OUT_DIR, "sql-embedding-sft-prompts-3.json")
json.dump(task, open(PROMPTS_FILE, "w"), indent=4)

In [12]:
OUT_DIR = os.path.join("chapter-4","results", "sql-embedding-sft-experiments", "3-shot")

PROMPTS_FILE = os.path.join(OUT_DIR, "sql-embedding-sft-prompts-3.json")

RESPONSES_FILE = os.path.join(OUT_DIR, "sql-embedding-sft-responses-3.txt")

run_llm(PROMPTS_FILE, RESPONSES_FILE, model="gpt-3.5-turbo")

100%|██████████| 1034/1034 [18:42<00:00,  1.09s/it] 


In [13]:
nest_asyncio.apply()

OUT_DIR = os.path.join("chapter-4","results", "sql-embedding-sft-experiments", "3-shot")

PROMPTS_FILE = os.path.join(OUT_DIR, "sql-embedding-sft-prompts-3.json")
RESPONSES_FILE = os.path.join(OUT_DIR, "sql-embedding-sft-responses-3.txt")
RESULTS_FILE = os.path.join(OUT_DIR,"sql-embedding-sft-results-3.json")

results = evaluate_spider(gold="benchmarks/spider/dev_gold.sql", pred=RESPONSES_FILE, db="benchmarks/spider/databases", table="benchmarks/spider/tables.json")

save_results(PROMPTS_FILE, RESPONSES_FILE, RESULTS_FILE, results)

  0%|          | 0/1 [00:00<?, ?it/s]


                     easy                 medium               hard                 extra                all                 
count                248                  446                  174                  166                  1034                
=====================   EXECUTION ACCURACY     =====================
execution            0.915                0.852                0.724                0.536                0.795               


## SQL Fine-Tuned Embedding Selector Spider 5-shot Experiment

In [14]:
prompt = prompt_factory(k_shot = 5, repr_type= "SQL", example_format= "QA", selector_type= "EMBEDSQL", embedding_model= EMBEDDING)(data=data, tokenizer="gpt-3.5-turbo")

In [15]:
# Format all Spider test questions for 5-Shot GPT-3.5-Turbo Text-to-SQL conversion
questions = list()

for i, question_json in enumerate(tqdm(getattr(data, "get_test_json")()), start=1):
    question_format = prompt.format(index=i,
                                    target=question_json,
                                    max_seq_len=2048,
                                    max_ans_len=200,
                                    scope_factor=1,
                                    cross_domain=False)
    questions.append(question_format)

100%|██████████| 1034/1034 [07:13<00:00,  2.38it/s]


In [16]:
# Define the task parameters for saving the questions for the Spider baseline experiment

args = {
    "data_type": "spider",
    "split": "test",
    "tokenizer": "gpt-3.5-turbo",
    "max_seq_length": 2048,
    "prompt_repr": "SQL",
    "k-shot": 5,
    "example_type": "QA",
    "selector_type": "EMBEDPRED",
    "embedding_model": EMBEDDING
}

# Define the task object for saving the questions for the Spider baseline experiment

task = {
        "args": args,
        "questions": questions
    }

OUT_DIR = os.path.join("chapter-4","results", "sql-embedding-sft-experiments", "5-shot")
os.makedirs(OUT_DIR, exist_ok =True)

PROMPTS_FILE = os.path.join(OUT_DIR, "sql-embedding-sft-prompts-5.json")
json.dump(task, open(PROMPTS_FILE, "w"), indent=4)

In [17]:
OUT_DIR = os.path.join("chapter-4","results", "sql-embedding-sft-experiments", "5-shot")

PROMPTS_FILE = os.path.join(OUT_DIR, "sql-embedding-sft-prompts-5.json")

RESPONSES_FILE = os.path.join(OUT_DIR, "sql-embedding-sft-responses-5.txt")

run_llm(PROMPTS_FILE, RESPONSES_FILE, model="gpt-3.5-turbo")

100%|██████████| 1034/1034 [14:55<00:00,  1.15it/s]


In [18]:
nest_asyncio.apply()

OUT_DIR = os.path.join("chapter-4","results", "sql-embedding-sft-experiments", "5-shot")

PROMPTS_FILE = os.path.join(OUT_DIR, "sql-embedding-sft-prompts-5.json")
RESPONSES_FILE = os.path.join(OUT_DIR, "sql-embedding-sft-responses-5.txt")
RESULTS_FILE = os.path.join(OUT_DIR,"sql-embedding-sft-results-5.json")

results = evaluate_spider(gold="benchmarks/spider/dev_gold.sql", pred=RESPONSES_FILE, db="benchmarks/spider/databases", table="benchmarks/spider/tables.json")

save_results(PROMPTS_FILE, RESPONSES_FILE, RESULTS_FILE, results)

  0%|          | 0/1 [00:00<?, ?it/s]


                     easy                 medium               hard                 extra                all                 
count                248                  446                  174                  166                  1034                
=====================   EXECUTION ACCURACY     =====================
execution            0.927                0.852                0.736                0.560                0.804               
